In [7]:
import requests
import pandas as pd

In [13]:
def get_coin_info(coin_id):
    base_url = "https://api.coingecko.com/api/v3"
    endpoint = f"/coins/{coin_id}"

    response = requests.get(base_url + endpoint)
    if response.status_code == 200:
        coin_data = response.json()
        return coin_data
    else:
        print("Error:", response.status_code)
        return None

In [20]:
data = get_coin_info('bitcoin')

In [21]:
data

{'id': 'bitcoin',
 'symbol': 'btc',
 'name': 'Bitcoin',
 'asset_platform_id': None,
 'platforms': {'': ''},
 'detail_platforms': {'': {'decimal_place': None, 'contract_address': ''}},
 'block_time_in_minutes': 10,
 'hashing_algorithm': 'SHA-256',
 'categories': ['Cryptocurrency', 'Layer 1 (L1)'],
 'public_notice': None,
 'additional_notices': [],
 'localization': {'en': 'Bitcoin',
  'de': 'Bitcoin',
  'es': 'Bitcoin',
  'fr': 'Bitcoin',
  'it': 'Bitcoin',
  'pl': 'Bitcoin',
  'ro': 'Bitcoin',
  'hu': 'Bitcoin',
  'nl': 'Bitcoin',
  'pt': 'Bitcoin',
  'sv': 'Bitcoin',
  'vi': 'Bitcoin',
  'tr': 'Bitcoin',
  'ru': 'Биткоин',
  'ja': 'ビットコイン',
  'zh': '比特币',
  'zh-tw': '比特幣',
  'ko': '비트코인',
  'ar': 'بيتكوين',
  'th': 'บิตคอยน์',
  'id': 'Bitcoin',
  'cs': 'Bitcoin',
  'da': 'Bitcoin',
  'el': 'Bitcoin',
  'hi': 'Bitcoin',
  'no': 'Bitcoin',
  'sk': 'Bitcoin',
  'uk': 'Bitcoin',
  'he': 'Bitcoin',
  'fi': 'Bitcoin',
  'bg': 'Bitcoin',
  'hr': 'Bitcoin',
  'lt': 'Bitcoin',
  'sl': 'Bitcoin

In [19]:
df = pd.DataFrame(data)
df.head()

ValueError: All arrays must be of the same length

In [10]:
url = "https://api.coingecko.com/api/v3/simple/price?ids=btc&vs_currencies=usd&include_market_cap=true&include_24hr_vol=true&include_24hr_change=true&include_last_updated_at=true&precision=0"

response = requests.get(url)
print(response)
data = response.json()

<Response [200]>


In [ ]:
price_usd = data['btc']['usd']
market_cap = data['btc']['usd_market_cap']
vol_24hr = data['btc']['usd_24h_vol']
change_24hr = data['btc']['usd_24h_change']
last_updated_at = data['btc']['last_updated_at']

print(f"Price USD: ${price_usd}")
print(f"Market Cap: ${market_cap}")
print(f"24hr Volume: ${vol_24hr}")
print(f"24hr Change: {change_24hr}%")
print(f"Last Updated At: {last_updated_at}")